# Simple usage example

## 1. Python

Compile two stencils and use them from python. The stencils are defined in the file `stencils.gt.py`

The following prints the help page for the `gtpyc` gtscript compiler:

In [1]:
!gtpyc --help

Usage: gtpyc [OPTIONS] INPUT_PATH

  GT4Py (GridTools for Python) stencil compiler.

  INPUT_PATH can be either a gtscript or python file or a python package.

Options:
  -b, --backend [debug|numpy|gtx86|gtmc|gtcuda]
                                  Choose a backend
  -o, --output-path DIRECTORY     output path for the compiled source.
  --bindings BINDINGS             Generate source code for language bindings.
                                  Which languages are available depends on the
                                  backend.

  -C, --compile-bindings          Compile bindings source code. Only has an
                                  effect in combination wih --bindings.

  -O, --option OTPION             Backend option (multiple allowed), format:
                                  -O key=value

  -E, --externals JSON            JSON string describing externals overrides
                                  for all stencils.

  -s, --silent                    suppress output.
  --h

### Compile the stencils
We use the backend "numpy" since currently only "numpy" and "debug" allow us to call the compiled stencils with numpy arrays.

In [2]:
!gtpyc -b "numpy" ./stencils.gt.py

reading input file stencils.gt.py
input file loaded as module <module 'stencils' from '/Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/stencils.gt.py'>
Found 2 stencils: "axpy", "dot".
Building stencil axpy
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/axpy.py
Building stencil dot
 -> /Users/ricoh/Code/gt4py-tutorial/gtpyc tutorial/examples/simple-stencils/dot.py


Now we can confirm that the two stencil modules are in the current directory:

In [3]:
!ls

1-gtscript-to-python.ipynb  _dot.cpython-37m-darwin.so
2-gtscript-to-c++.ipynb     axpy.py
3-c++-python-bindings.ipynb dot.py
_axpy.cpython-37m-darwin.so stencils.gt.py


### Use the stencils in python

In [4]:
import numpy

The stencils get compiled to a callable StencilObject class, which we can import and instanciate.

In [5]:
from dot import dot as dot_cls
from axpy import axpy as axpy_cls
dot = dot_cls()
axpy = axpy_cls()

Now we can use the stencil object instances like any function. Note, that we have to pass it the `origin` keyword argument, because we are using `numpy.array` inputs.
Using numpy arrays also leads to a warning being issued that this is not (yet) supported by all backends.

In [6]:
x = numpy.array([[[1, 2], [1, 4]]], dtype=numpy.float64)
y = numpy.array([[[2, 3], [4, 4]]], dtype=numpy.float64)
res = numpy.empty(shape=(1, 2, 2), dtype=numpy.float64)

dot(x, y, res, origin=(0, 0, 0))
axpy(x, y, alpha=2., origin=(0, 0, 0))

/Users/ricoh/Code/gt4py/src/gt4py/stencil_object.py:198: RuntimeWarning: NumPy ndarray passed as field. This is discouraged and only works with constraints and only for certain backends.
  RuntimeWarning,


Let's confirm the results of the `dot` stencil, which parallelly multiplies vertical columns and then accumulates them in the last layer.
```
x[i, j, :] * y[i, j, :] -> dot[i, j, -1]
```

In [7]:
assert res[0, 0, -1] == numpy.dot([1., 2.], [2., 3.])
assert res[0, 1, -1] == numpy.dot([1., 4.], [4., 4.])
res[:, :, -1]

array([[ 8., 20.]])

Now we check the `axpy` results, which again treats the input array as a bunch of (vertical) vectors and operates on them in parallel.
```
x * alpha + y -> y
```

In [8]:
assert numpy.all(y[0, 0, :] == numpy.array([1., 2.]) * 2. + numpy.array([2., 3.]))
assert numpy.all(y[0, 1, :] == numpy.array([1., 4.]) * 2. + numpy.array([4., 4.]))
y

array([[[ 4.,  7.],
        [ 6., 12.]]])

If you inspect the generated python modules you will notice that they in turn do import from `gt4py`. However since they can automatically be generated it is possible to formally keep the dependency outside versioned code, simply by providing a script / build system that regenerates the stencil modules on demand. This could even happen automatically on catching an ImportError.

### Bonus: use the same gtscript file directly from python
This now requires explicitly importing from gt4py in order to allow importing gtscript files.

In [9]:
from gt4py import gtsimport
gtsimport.install(search_path=["."])

In [10]:
from stencils import dot

In [11]:
dot(x, y, res, origin=(0, 0, 0))
res

array([[[ 4., 18.],
        [ 6., 54.]]])

### Cleanup
Disable the following cell (or outcomment specific files) to keep generated files around.

In [1]:
import pathlib
import shutil
cwd = pathlib.Path('.')
axpy_mod = cwd / 'axpy.py'
dot_mod = cwd / 'dot.py'
axpy_ext = cwd / '_axpy.so'
dot_ext = cwd / '_dot.so'
pycache = cwd / '__pycache__'
axpy_src = cwd / 'src_axpy'
dot_src = cwd / 'src_dot'
files = [axpy_mod, dot_mod, axpy_ext, dot_ext]
dirs = [pycache, axpy_src, dot_src]

for file in files:
    if file.exists():
        file.unlink()
for folder in dirs:   
    if folder.exists():
        shutil.rmtree(folder)